Script to Clean up Lab Automation Data before Spotfire Import - 

Script 1 is for 1 single file (latest file)

Script 2 is for all unprocessed files in the folder

# Script 1

In [ ]:

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pathlib


# * means all if need specific format then *.csv
list_of_files = glob.glob('C:/Datafiles/AMC1300/*.txt') 
latest_file = max(list_of_files, key=os.path.getctime)
print (latest_file)
File_name=os.path.basename(latest_file)
print(File_name)
#Remove the .txt extension
File_name = File_name[:-4]

path = pathlib.Path('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
re_search = re.search('..Temporary..', File_name)


 
if (path.is_file() == False and re_search == None):
#Read the file
        print("Inside the IF")
        df = pd.read_csv(latest_file,sep='\t',delimiter='\t')
        df=df.set_index('Meas_Name')
        df=df.drop(['RunComment','STRING','TEST_ID'])
        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+File_name+'.csv')
     

# Script 2 - Removes trash columns from data log and inserts type row


In [3]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os , time
import glob
import pathlib
import re
list_of_files_raw_noext=[]
#list_of_files_raw = glob.glob('C:/Datafiles/AMC1300/*.txt') 
No_of_files_processed=0
No_of_files_skipped=0
#PATH_FOR_CLEAN_DATALOG= 'C:/Datafiles/AMC1300/Cleanedup/'
#PATH_FOR_RAW_DATALOG= 'C:/Datafiles/AMC1300/'

#Get list of files in datalog folder
list_of_files_raw = glob.glob('//Dtfr-ctpb1h43b/amc1300/*.txt')

#initialize paths
PATH_FOR_CLEAN_DATALOG= '//Dtfr-ctpb1h43b/amc1300/Cleanedup/'
PATH_FOR_RAW_DATALOG= '//Dtfr-ctpb1h43b/amc1300/'
##BASE_FILE_TIME = 1519948481.7437975


#//Dtfr-ctpb1h43b/amc1300
#AMC13XX (file://Dtfr-ctpb1h43b/amc13xx/)
#Cleanedup (file://Dtfr-ctpb1h43b/amc13xx/Cleanedup)
list_of_files_clean = glob.glob('//Dtfr-ctpb1h43b/amc1300/Cleanedup/*.csv') # * means all if need specific format then *.csv
latest_file = max(list_of_files_clean, key=os.path.getctime)
#print ('Latest clean file is ' + latest_file)
BASE_FILE_TIME = os.path.getmtime(latest_file)

for file in list_of_files_raw:
    
    re_search_bak = re.search('..bak', file) # search for .bak in filename
    file_noext =os.path.basename(file[:-4]) #remove .ext from file name
    list_of_files_raw_noext.append(file)
    path = pathlib.Path(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
    
    
    re_search = re.search('..Temporary..', file_noext)
    
    if (path.is_file() == False and re_search==None and re_search_bak==None and os.path.getmtime(file)> BASE_FILE_TIME ):
        
        print("Processing file..."+os.path.basename(file))
        #print("last modified: %s" % time.ctime(os.path.getmtime(file)))
        #print("last modified: %s" % os.path.getmtime(file))
        #Read the file
        df = pd.read_csv(PATH_FOR_RAW_DATALOG+file_noext+'.txt',sep='\t',delimiter='\t')
        
        #set Meas_Name as the DF index
        #df=df.set_index('Device Descriptor')
        df['index_col'] = df.index
        #Rows to be removed (dropped), row 0 and 1
        df=df.drop([0,1])
      
         #columns to be removed (dropped)

        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        #df = df.rename(columns={col: col.split('(')[0] for col in df.columns})
        
        
        type_row=[]    
        for column in df:
            #print(column)
            #print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append(3.05) #just insert a real num in the first row so that spotfire detects it
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
        try:
            #Make a data frame with df.columns as index and type_row as column
            df2 = pd.Series(type_row, index=[df.columns])
            #append df2 transposed to df, gets type_as the last row
            df2 = df.append(df2, ignore_index=True)
            #swap last row(type row ) and first row
            last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
            df2.iloc[-1],df2.iloc[0] = first_row,last_row
        
            df2.to_csv(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
            
        except:
            print("Not a valid file "+file_noext+'.txt')
            
   





Processing file...AMC1300_14150468.txt


In [28]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os , time
import glob
import pathlib
import re
list_of_files_raw_noext=[]

PATH_FOR_CLEAN_DATALOG= '//Dtfr-ctpb1h43b/amc1300/Cleanedup/'
PATH_FOR_RAW_DATALOG= '//Dtfr-ctpb1h43b/amc1300/'

#Get list of files in datalog folder
list_of_files_raw = glob.glob(PATH_FOR_RAW_DATALOG+'*.txt')
list_of_files_clean = glob.glob(PATH_FOR_CLEAN_DATALOG+'*.csv')
latest_file = max(list_of_files_clean, key=os.path.getctime)
BASE_FILE_TIME = os.path.getmtime(latest_file)

for file in list_of_files_raw:
    
    re_search_bak = re.search('..bak', file) # search for .bak in filename
    file_noext =os.path.basename(file[:-4]) #remove .ext from file name
    list_of_files_raw_noext.append(file)
    path = pathlib.Path(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
    re_search = re.search('..Temporary..', file_noext)
    if (path.is_file() == False and re_search==None and 
        re_search_bak==None and os.path.getmtime(file)> BASE_FILE_TIME ):
        print("Processing file..."+os.path.basename(file))
        #Read the file
        df = pd.read_csv(PATH_FOR_RAW_DATALOG+file_noext+'.txt',sep='\t',delimiter='\t')
        df['index_col'] = df.index
        #Rows to be removed (dropped), row 0 and 1
        df=df.drop([0,1])
        #columns to be removed (dropped)
        df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        #df = df.rename(columns={col: col.split('(')[0] for col in df.columns})
        type_row=[]    
        for column in df:
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
    
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append(3.05) #Dummy real for spotfire dtype detection to work
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        break       
        
        try:
            #Make a data frame with df.columns as index and type_row as column
            df2 = pd.Series(type_row, index=[df.columns])
            #append df2 transposed to df, gets type_as the last row
            df2 = df.append(df2, ignore_index=True)
            #swap last row(type row ) and first row
            last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
            df2.iloc[-1],df2.iloc[0] = first_row,last_row
            df2.to_csv(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
            
        except:
            print("Not a valid file "+file_noext+'.txt')

Processing file...AMC1300_14150468.txt
Processing file...AMC1300_143352425.txt


C:\Users\a0406675\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (0,1,3,4,5,9,12,13,14,19,20,21,22,23,24,25,26,27,29,30,31,32,33,34,35,36,37,38,39,44,45,46,47,48,49,50,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Code to fuse temporary .bak and header file.

1.Find the temp file pair which are newer than the last clean file
2.extract the header and remove NFC
3.Add the header as the first row 

Try to make the script efficient by making a list of "new" raw files only  #fail


In [1]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os , time
import glob
import pathlib
import re
list_of_files_raw_noext=[]
No_of_files_processed=0
No_of_files_skipped=0

#initialize paths
PATH_FOR_CLEAN_DATALOG= '//Dtfr-ctpb1h43b/amc1300/Cleanedup/'
PATH_FOR_RAW_DATALOG= '//Dtfr-ctpb1h43b/amc1300/'
##BASE_FILE_TIME = 1519948481.7437975


def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

#Get list of files in datalog folder
list_of_files_raw = glob.glob(PATH_FOR_RAW_DATALOG+'*.txt')
list_of_files_clean = glob.glob(PATH_FOR_CLEAN_DATALOG+'*.csv')

latest_file_clean = max(list_of_files_clean, key=os.path.getctime)
#print ('Latest clean file is ' + os.path.basename(latest_file_clean))
latest_clean_file_noext =os.path.basename(os.path.basename(latest_file_clean)[:-4])
#print ('Latest clean file noext is ' + latest_clean_file_noext)
last_processed_file=find(latest_clean_file_noext+'.txt',PATH_FOR_RAW_DATALOG)
print('Last Processed file is '+last_processed_file)
list_of_raw_files_unprocessed=[]

#BASE_FILE_TIME = os.path.getmtime(latest_file_clean)


#list_of_files_raw.sort(key=os.path.getmtime)
#list_of_files_raw.reverse()

for file in list_of_files_raw:
    if (os.path.getmtime(file)>os.path.getmtime(last_processed_file)):
        re_search_bak = re.search('..bak', file)
        file_noext =os.path.basename(file[:-4]) #remove .ext from file name
        re_search = re.search('..Temporary..', file_noext)
        if (re_search==None and re_search_bak==None):
            list_of_raw_files_unprocessed.append(file)
            print('File '+file+' Is added for processing')
    else :
        break
                    
            
#list_of_raw_files_unprocessed.reverse()            
for file in list_of_raw_files_unprocessed:
    file_noext =os.path.basename(file[:-4])
    print("Processing file..."+file_noext+".txt")
        #print("last modified: %s" % time.ctime(os.path.getmtime(file)))
        #print("last modified: %s" % os.path.getmtime(file))
        #Read the file
    #df = pd.read_csv(PATH_FOR_RAW_DATALOG+file_noext+'.txt',sep='\t',delimiter='\t')  
    df = pd.read_csv(file,sep='\t',delimiter='\t')
    df['index_col'] = df.index
        #Rows to be removed (dropped), row 0 and 1
    df=df.drop([0,1])
      
         #columns to be removed (dropped)

    df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        #df = df.rename(columns={col: col.split('(')[0] for col in df.columns})
        
        
    type_row=[]    
    for column in df:
            #print(column)
            #print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append(3.05) #just insert a real num in the first row so that spotfire detects it
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
    try:
            #Make a data frame with df.columns as index and type_row as column
            df2 = pd.Series(type_row, index=[df.columns])
            #append df2 transposed to df, gets type_as the last row
            df2 = df.append(df2, ignore_index=True)
            #swap last row(type row ) and first row
            last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
            df2.iloc[-1],df2.iloc[0] = first_row,last_row
        
            df2.to_csv(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
            
    except:
            print("Not a valid file "+file_noext+'.txt')

Last Processed file is //Dtfr-ctpb1h43b/amc1300/AMC1300_72726868.txt


# All code below is invalid and is used for debug

In [ ]:
 #columns to be removed (dropped)

df=df.drop(['Sweep Version', 'Test Area','Package Type','Product Revision','Test_Parameter',
            'Block ID','Channel','Test Type','Start time','Lot','Station Name','SCM',
            'Upper Limit','Lower Limit','Nominal Value','End Time','Execution Duration (ms)',
            'Loop Count','Run Mode Name','UserID','Comment'],axis=1)
        df = df.rename(columns={col: col.split('(')[0] for col in df.columns})      
# Type row code
        
        # write cleaned up dataframe to .csv file of the same name
        df2.to_csv('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.csv')
        print ('Saved as...'+file_noext+'.csv')
#         writer = pd.ExcelWriter('C:/Datafiles/AMC1300/Cleanedup/'+file_noext+'.xlsx')
#         df2.to_excel(writer,'Sheet1')
#         writer.save()
#         print ('Saved as...'+file_noext+'.xlsx')
        #update processed file counter
        No_of_files_processed+=1
       
    else :
        print ('Skipping file...',os.path.basename(file)) 
        No_of_files_skipped+=1     
            
            # import pdb; pdb.set_trace()     # Break point for debug
print(No_of_files_processed,'files processed')  
print (No_of_files_skipped,'files skipped')


#for file in list_of_files_processed:
 #   file =os.path.basename(file[:-4])
    #print(file)
    #list_of_files_processed_noext.append(file)



##for file in list_of_files_raw:
 ##   file =file[:-4]
    #print(file)
   ## list_of_files_raw_noext.append(file)
    
##for file in list_of_files_processed:
  ##  file =file[:-4]
    #print(file)
    ##list_of_files_processed_noext.append(file)    
    
##print(list_of_files_raw_noext)  
#print(list_of_files_processed_noext)   


#s = set(list_of_files_processed_noext)
#list_of_files_notprocessed_noext = [x for x in list_of_files_raw_noext if x not in s]
#print(list_of_files_notprocessed_noext)  




#Type Row Code


        
        #print(df.dtypes)
        
        type_row=[]    
        for column in df:
            #print(column)
            ##print(df[column].unique())
            #print('Column index is',df.columns.get_loc(column))
            Column_idx=df.columns.get_loc(column)
            for unique_val in df[column].unique():
                if(pd.isnull(unique_val)==True):
                #print("this element is NaN")
                  pass
                else:
                    try:
                        float(unique_val)
                        isnumber= True
                        type_row.append('FLOAT')
                        #print(column ,'is a Number')
                        break 
                    except:
                        Isnumber = False   
                        type_row.append('STRING')
                        #print( column,'is a String')
                        break       
        
        
        #Make a data frame with df.columns as index and type_row as column
        df2 = pd.Series(type_row, index=[df.columns])
        #append df2 transposed to df, gets type_as the last row
        df2 = df.append(df2, ignore_index=True)
        #swap last row(type row ) and first row
        last_row, first_row = df2.iloc[-1].copy(), df2.iloc[0].copy()
        df2.iloc[-1],df2.iloc[0] = first_row,last_row



In [ ]:
%run BenchDatalog_Cleanup.py



In [ ]:
%run BenchDatalog_Cleanup.py

In [25]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os , time
import glob
import pathlib
import re
list_of_files_raw_noext=[]

PATH_FOR_CLEAN_DATALOG= '//Dtfr-ctpb1h43b/amc1300/Cleanedup/'
PATH_FOR_RAW_DATALOG= '//Dtfr-ctpb1h43b/amc1300/'

#Get list of files in datalog folder
list_of_files_raw = glob.glob(PATH_FOR_RAW_DATALOG+'*.txt')
list_of_files_clean = glob.glob(PATH_FOR_CLEAN_DATALOG+'*.csv')
latest_file = max(list_of_files_clean, key=os.path.getctime)
BASE_FILE_TIME = os.path.getmtime(latest_file)
temp_files=[]


list_of_files_raw.sort(key=os.path.getmtime)
temp_files=list_of_files_raw[-2:]
df = pd.read_csv(temp_files[-2],sep='\t',delimiter='\t')


for file in list_of_files_raw:
    
    re_search_bak = re.search('..bak', file) # search for .bak in filename
    file_noext =os.path.basename(file[:-4]) #remove .ext from file name
    list_of_files_raw_noext.append(file)
    path = pathlib.Path(PATH_FOR_CLEAN_DATALOG+file_noext+'.csv')
    re_search = re.search('..Temporary..', file_noext)
    if (path.is_file() == False and re_search==True
         and os.path.getmtime(file)> BASE_FILE_TIME ):
        print("Processing file..."+os.path.basename(file))

In [26]:
df

,Device Descriptor,Sweep Version,Product Name,Test Area,Package Type,Product Revision,Test Program Name,Meas_Name,Test_Parameter,Meas_Value,...,SFC - Source,SFC - SCR_ON,NFC - Vpp,NFC - F_Sample,NFC - Bandwidth_INP_Freq(Hz),NFC - Amp,NFC - CLK,NFC - Fin,NFC - DC_INPUT_FC,NFC - INL_DC_INPUT_MEAS_FC(V)


In [23]:
temp_files[-2]


'//Dtfr-ctpb1h43b/amc1300/AMC1300_143352425- Temporary Header File.txt'